## goal
### - 将sensor按照census tract聚合，每个census tract每天的值取均值代表这个census tract在dateX的值
### - census package - https://pygis.io/docs/d_access_census.html

In [49]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from census import Census
from us import states
import os
import requests

In [77]:
c = Census("59c822f1e8302f7e57bbdd6d0cf1fd7649de1f0e")
API_KEY = '59c822f1e8302f7e57bbdd6d0cf1fd7649de1f0e'

In [65]:
# url_profile  = 'https://api.census.gov/data/2021/acs/acs5/profile/variables.json'
# resp_profile = requests.get(url_profile).json()
# variables_profile = resp_profile['variables'].keys()
# print("Profile tables 示例变量：", sorted(list(variables_profile)))

In [87]:
# Assign variable codes and their corresponding labels
profile_vars = ('GEO_ID',
             'NAME', 
             'DP05_0001E', # total population
             'DP05_0002E', # male
             'DP05_0003E', # female
             'DP05_0070E', #  !!Total:!!Hispanic or Latino
             'DP05_0077E', #  !!Total:!!Not Hispanic or Latino:!!Population of one race:!!White alone
             'DP05_0078E', #  !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Black or African American alone
             'DP05_0079E', #  !!Total:!!Not Hispanic or Latino:!!Population of one race:!!American Indian and Alaska Native alone
             'DP05_0080E', #  !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Asian alone
             'DP05_0081E', #  !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Native Hawaiian and Other Pacific Islander alone
             'DP05_0082E', #  !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Some Other Race alone
             'S1101_C01_001E', # Total households
             'S1101_C01_002E', # Average household size
             'S1101_C01_005E', # Estimate!!Total!!AGE OF OWN CHILDREN!!Households with own children of the householder under 18 years
             'B25034_001E', # total building structures
             'B25034_002E', # Estimate!!Total:!!Built 2014 or later
             'B25034_003E', # Estimate!!Total:!!Built 2010 to 2013
             'B25034_004E', # Estimate!!Total:!!Built 2000 to 2009
             'B25034_005E', # Estimate!!Total:!!Built 1990 to 1999
             'B25034_006E', # Estimate!!Total:!!Built 1980 to 1989
             'B25034_007E', # Estimate!!Total:!!Built 1970 to 1979
             'B25034_008E', # Estimate!!Total:!!Built 1960 to 1969
             'B25034_009E', # Estimate!!Total:!!Built 1950 to 1959
             'B25034_010E', # Estimate!!Total:!!Built 1940 to 1949
             'B25034_011E', # Estimate!!Total:!!Built 1939 or earlier
             'S1501_C01_012E', # Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree
            )
labels =    ('GEOID_str',
             'NAME', 
             'pop_total', 
             'male',
             'female',
             'hispanic',
             'nonH_white', 
             'nonH_black', 
             'nonH_native',
             'nonH_asian', 
             'nonH_pacific',
             'hh_count',
             'hh_avg_size',
             'child_count',
             'building_count',
             'b_2014_',
             'b_2010_2013',
             'b_2000_2009',
             'b_1990_1999',
             'b_1980_1989',
             'b_1970_1979',
             'b_1960_1969',
             'b_1950_1959',
             'b_1940_1949',
             'b__1939'
            )

In [95]:
# 2021
base = 'https://api.census.gov/data/2021/acs'

state, county = states.CA.fips, '037'
geo = f'for=tract:*&in=state:{state}%20county:{county}&key={API_KEY}'

# 1) Profile: DP05_*
prof_vars = ['GEO_ID','NAME','DP05_0001E','DP05_0002E','DP05_0003E',
             'DP05_0070E','DP05_0077E','DP05_0078E','DP05_0079E',
             'DP05_0080E','DP05_0081E','DP05_0082E']
url_prof = f'{base}/acs5/profile?get={",".join(prof_vars)}&{geo}'
r1 = requests.get(url_prof)
print('Profile status:', r1.status_code)      # 应该 200
df_prof_2021 = pd.DataFrame(r1.json()[1:], columns=r1.json()[0])

# 2) Subject: S1101_… 和 S1501_…
subj_vars = ['GEO_ID','NAME','S1101_C01_001E','S1101_C01_002E','S1101_C01_005E',
             'S1501_C01_012E']
url_subj = f'{base}/acs5/subject?get={",".join(subj_vars)}&{geo}'
r2 = requests.get(url_subj)
print('Subject status:', r2.status_code)
df_subj_2021 = pd.DataFrame(r2.json()[1:], columns=r2.json()[0])

# 3) Detail: B25034_001E–B25034_011E
det_vars = ['GEO_ID', 'NAME', 'B25034_001E','B25034_002E', 'B25034_003E', 'B25034_004E', 'B25034_005E', 'B25034_006E', 'B25034_007E',\
           'B25034_008E', 'B25034_009E', 'B25034_010E', 'B25034_011E']
url_det = f'{base}/acs5?get={",".join(det_vars)}&{geo}'
r3 = requests.get(url_det)
print('Detail status:', r3.status_code)
df_det_2021 = pd.DataFrame(r3.json()[1:], columns=r3.json()[0])

# 最后：按 GEO_ID 合并
df_2021 = df_prof_2021.merge(df_subj_2021, on='GEO_ID')\
            .merge(df_det_2021,  on='GEO_ID')

Profile status: 200
Subject status: 200
Detail status: 200


In [97]:
# 2022
base = 'https://api.census.gov/data/2022/acs'

state, county = states.CA.fips, '037'
geo = f'for=tract:*&in=state:{state}%20county:{county}&key={API_KEY}'

# 1) Profile: DP05_*
prof_vars = ['GEO_ID','NAME','DP05_0001E','DP05_0002E','DP05_0003E',
             'DP05_0070E','DP05_0077E','DP05_0078E','DP05_0079E',
             'DP05_0080E','DP05_0081E','DP05_0082E']
url_prof = f'{base}/acs5/profile?get={",".join(prof_vars)}&{geo}'
r1 = requests.get(url_prof)
print('Profile status:', r1.status_code)      # 应该 200
df_prof_2022 = pd.DataFrame(r1.json()[1:], columns=r1.json()[0])

# 2) Subject: S1101_… 和 S1501_…
subj_vars = ['GEO_ID','NAME','S1101_C01_001E','S1101_C01_002E','S1101_C01_005E',
             'S1501_C01_012E']
url_subj = f'{base}/acs5/subject?get={",".join(subj_vars)}&{geo}'
r2 = requests.get(url_subj)
print('Subject status:', r2.status_code)
df_subj_2022 = pd.DataFrame(r2.json()[1:], columns=r2.json()[0])

# 3) Detail: B25034_001E–B25034_011E
det_vars = ['GEO_ID', 'NAME', 'B25034_001E','B25034_002E', 'B25034_003E', 'B25034_004E', 'B25034_005E', 'B25034_006E', 'B25034_007E',\
           'B25034_008E', 'B25034_009E', 'B25034_010E', 'B25034_011E']
url_det = f'{base}/acs5?get={",".join(det_vars)}&{geo}'
r3 = requests.get(url_det)
print('Detail status:', r3.status_code)
df_det_2022 = pd.DataFrame(r3.json()[1:], columns=r3.json()[0])

# 最后：按 GEO_ID 合并
df_2022 = df_prof_2022.merge(df_subj_2022, on='GEO_ID')\
            .merge(df_det_2022,  on='GEO_ID')

Profile status: 200
Subject status: 200
Detail status: 200


In [120]:
df_2021['state']  = df_2021['state'].astype(str).str.zfill(2)
df_2021['county'] = df_2021['county'].astype(str).str.zfill(3)
df_2021['tract']  = df_2021['tract'].astype(str).str.zfill(6)
df_2021['GEOID']  = df_2021['state'] + df_2021['county'] + df_2021['tract']

df_2022['state']  = df_2022['state'].astype(str).str.zfill(2)
df_2022['county'] = df_2022['county'].astype(str).str.zfill(3)
df_2022['tract']  = df_2022['tract'].astype(str).str.zfill(6)
df_2022['GEOID']  = df_2022['state'] + df_2022['county'] + df_2022['tract']

df_2021.to_csv('../Data/Output/census_data_extraction_2021.csv')
df_2022.to_csv('../Data/Output/census_data_extraction_2022.csv')

df_2022.head(1)

,Unnamed: 0,GEO_ID,NAME_x,DP05_0001E,DP05_0002E,DP05_0003E,DP05_0070E,DP05_0077E,DP05_0078E,DP05_0079E,...,B25034_006E,B25034_007E,B25034_008E,B25034_009E,B25034_010E,B25034_011E,state,county,tract,GEOID
0,0,1400000US06037101110,Census Tract 1011.10; Los Angeles County; Cali...,4014,1905,2109,0,174,2932,2236,...,79,119,213,516,223,328,06,037,101110,06037101110


In [105]:
# census tract shapefile
ca_tract = gpd.read_file("../Data/ACS/tl_2021_06_tract/tl_2021_06_tract.shp")

# https://spatialreference.org/ref/epsg/?search=4326
ca_tract = ca_tract.to_crs(epsg = 4326) # EPSG:4326: WGS 84
ca_tract["GEOID_int"] = ca_tract["GEOID"].astype("Int64")
print(ca_tract.head(2))
print('Shape: ', ca_tract.shape)
print("\nThe shapefile projection is: {}".format(ca_tract.crs))

  STATEFP COUNTYFP TRACTCE        GEOID     NAME              NAMELSAD  MTFCC  \
0      06      085  504321  06085504321  5043.21  Census Tract 5043.21  G5020   
1      06      085  504410  06085504410  5044.10  Census Tract 5044.10  G5020   

  FUNCSTAT    ALAND  AWATER     INTPTLAT      INTPTLON  \
0        S  1450237       0  +37.3931319  -121.8651427   
1        S  1102136       0  +37.4093719  -121.8788884   

                                            geometry   GEOID_int  
0  POLYGON ((-121.87556 37.39924, -121.87535 37.3...  6085504321  
1  POLYGON ((-121.88886 37.40758, -121.88576 37.4...  6085504410  
Shape:  (9129, 14)

The shapefile projection is: EPSG:4326


In [109]:
ca_tract.iloc[0]

STATEFP                                                     06
COUNTYFP                                                   085
TRACTCE                                                 504321
GEOID                                              06085504321
NAME                                                   5043.21
NAMELSAD                                  Census Tract 5043.21
MTFCC                                                    G5020
FUNCSTAT                                                     S
ALAND                                                  1450237
AWATER                                                       0
INTPTLAT                                           +37.3931319
INTPTLON                                          -121.8651427
geometry     POLYGON ((-121.875559 37.39924, -121.875352 37...
GEOID_int                                           6085504321
Name: 0, dtype: object

# where are sensors: descriptive table

In [114]:
gdf_pa = gpd.read_file("../Data/Output/PurpleAir_after_Calibration.shp")
df_2021 = pd.read_csv('../Data/Output/census_data_extraction_2021.csv')
df_2022 = pd.read_csv('../Data/Output/census_data_extraction_2022.csv')


gdf_pa['date'] = pd.to_datetime(gdf_pa['date'])
mask = (gdf_pa['date'] >= '2020-12-01') & (gdf_pa['date'] <= '2022-12-01')
gdf_pa = gdf_pa.loc[mask].copy()

# join sensor data with census geoid
gdf_pa = gdf_pa.to_crs(ca_tract.crs)
gdf_pa = gpd.sjoin(gdf_pa, ca_tract[['GEOID','geometry']], how = 'left', predicate = 'within')

# define year period:
def assign_year(d):
    return 2021 if d <= pd.Timestamp('2021-12-31') else 2022
gdf_pa['year_period'] = gdf_pa['date'].apply(assign_year)

In [134]:
gdf_pa.columns

Index(['Unnamed_ 0', 'sensor_id', 'date', 'time_stamp', 'rssi', 'uptime_x',
       'pa_latency', 'humidity_a', 'humidity_b', 'temperatur', 'temperat_1',
       'pressure_a', 'pressure_b', 'voc_a', 'voc_b', 'pm2.5_atm_',
       'pm2.5_at_1', 'pm10.0_atm', 'pm10.0_a_1', 'sensor_ind', 'date_creat',
       'location_t', 'model', 'uptime_y', 'position_r', 'latitude',
       'longitude', 'altitude', 'confidence', 'pm25_avg', 'PurpleAir_',
       'PurpleAi_1', 'PurpleAi_2', 'PA_PM25_ca', 'PA_PM25__1', 'diff_05',
       'diff_025', 'geometry', 'index_right', 'GEOID', 'year_period'],
      dtype='object')

In [136]:
gdf_pa.columns = ['Unnamed_ 0', 'sensor_id', 'date', 'time_stamp', 'rssi', 'uptime_x',
               'pa_latency', 'humidity_a', 'humidity_b', 'temperatur', 'temperat_1',
               'pressure_a', 'pressure_b', 'voc_a', 'voc_b', 'pm2.5_atm_',
               'pm2.5_at_1', 'pm10.0_atm', 'pm10.0_a_1', 'sensor_ind', 'date_creat',
               'location_t', 'model', 'uptime_y', 'position_r', 'latitude',
               'longitude', 'altitude', 'confidence', 'pm25_avg', 'PA_PM25',
               'PA_RH', 'PA_T', 'PA_PM25cali_025', 'PA_PM25cali_05', 'diff_05',
               'diff_025', 'geometry', 'index_right', 'GEOID', 'year_period']

In [138]:
# join census to sensor
gdf_pa_2021 = gdf_pa[gdf_pa['year_period'] == 2021]
gdf_pa_2022 = gdf_pa[gdf_pa['year_period'] == 2022]
sensor_2021_census = gdf_pa_2021.merge(df_2021, on = 'GEOID', how = 'left')
sensor_2022_census = gdf_pa_2022.merge(df_2022, on = 'GEOID', how = 'left')
sensor_2021_census.columns

Index(['Unnamed_ 0', 'sensor_id', 'date', 'time_stamp', 'rssi', 'uptime_x',
       'pa_latency', 'humidity_a', 'humidity_b', 'temperatur', 'temperat_1',
       'pressure_a', 'pressure_b', 'voc_a', 'voc_b', 'pm2.5_atm_',
       'pm2.5_at_1', 'pm10.0_atm', 'pm10.0_a_1', 'sensor_ind', 'date_creat',
       'location_t', 'model', 'uptime_y', 'position_r', 'latitude',
       'longitude', 'altitude', 'confidence', 'pm25_avg', 'PA_PM25', 'PA_RH',
       'PA_T', 'PA_PM25cali_025', 'PA_PM25cali_05', 'diff_05', 'diff_025',
       'geometry', 'index_right', 'GEOID', 'year_period', 'Unnamed: 0',
       'GEO_ID', 'NAME_x', 'DP05_0001E', 'DP05_0002E', 'DP05_0003E',
       'DP05_0070E', 'DP05_0077E', 'DP05_0078E', 'DP05_0079E', 'DP05_0080E',
       'DP05_0081E', 'DP05_0082E', 'state_x', 'county_x', 'tract_x', 'NAME_y',
       'S1101_C01_001E', 'S1101_C01_002E', 'S1101_C01_005E', 'S1501_C01_012E',
       'state_y', 'county_y', 'tract_y', 'NAME', 'B25034_001E', 'B25034_002E',
       'B25034_003E', 'B250

In [ ]:
sensor_2021_census['date'] = pd.to_datetime(sensor_2021_census['date'])
mask_2021 = (sensor_2021_census['date'] >= '2020-12-01') & (sensor_2021_census['date'] <= '2021-12-31')
s21 = sensor_2021_census.loc[mask_2021].copy()
mask_2022 = (sensor_2021_census['date'] >= '2022-01-01') & (sensor_2021_census['date'] <= '2022-12-01')
s22 = sensor_2021_census.loc[mask_2022].copy()

In [ ]:
# how many active days for each sensor
sensor_days_year = s21.groupby('sensor_id')['date'] # --------------- [change s21]
      .agg(lambda x: x.dt.date.nunique())
      .reset_index(name = 'active_days')

# active rate = active days/ one year
period_year_days = (pd.Timestamp('2021-12-31') - pd.Timestamp('2020-12-01')).days + 1 # ---------------- [change date]
sensor_days_year['active_pct'] = sensor_days_year['active_days'] / period_year_days * 100

feature_cols = [
     'DP05_0001E', # total population
     'DP05_0002E', # male
     'DP05_0003E', # female
     'DP05_0070E', #  !!Total:!!Hispanic or Latino
     'DP05_0077E', #  !!Total:!!Not Hispanic or Latino:!!Population of one race:!!White alone
     'DP05_0078E', #  !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Black or African American alone
     'DP05_0079E', #  !!Total:!!Not Hispanic or Latino:!!Population of one race:!!American Indian and Alaska Native alone
     'DP05_0080E', #  !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Asian alone
     'DP05_0081E', #  !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Native Hawaiian and Other Pacific Islander alone
     'DP05_0082E', #  !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Some Other Race alone
     'S1101_C01_001E', # Total households
     'S1101_C01_002E', # Average household size
     'S1101_C01_005E', # Estimate!!Total!!AGE OF OWN CHILDREN!!Households with own children of the householder under 18 years
     'B25034_001E', # total building structures
     'B25034_002E', # Estimate!!Total:!!Built 2014 or later
     'B25034_003E', # Estimate!!Total:!!Built 2010 to 2013
     'B25034_004E', # Estimate!!Total:!!Built 2000 to 2009
     'B25034_005E', # Estimate!!Total:!!Built 1990 to 1999
     'B25034_006E', # Estimate!!Total:!!Built 1980 to 1989
     'B25034_007E', # Estimate!!Total:!!Built 1970 to 1979
     'B25034_008E', # Estimate!!Total:!!Built 1960 to 1969
     'B25034_009E', # Estimate!!Total:!!Built 1950 to 1959
     'B25034_010E', # Estimate!!Total:!!Built 1940 to 1949
     'B25034_011E', # Estimate!!Total:!!Built 1939 or earlier
     'S1501_C01_012E', # Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree
]
sensor_features_year = sensor_2021_census[feature_cols]

sensor_panel_year = sensor_days_year.merge(
    sensor_features_2021,
    on = 'sensor_id',
    how = 'left'
)

# 7. 衍生比率指标
sensor_panel_2021['pop_total']    = sensor_panel_2021['DP05_0001E']
sensor_panel_2021['pct_hispanic'] = sensor_panel_2021['DP05_0070E'] / sensor_panel_2021['DP05_0001E'] * 100
sensor_panel_2021['pct_white']    = sensor_panel_2021['DP05_0077E'] / sensor_panel_2021['DP05_0001E'] * 100
sensor_panel_2021['pct_bach']     = sensor_panel_2021['S1501_C01_012E'] / sensor_panel_2021['DP05_0001E'] * 100
sensor_panel_2021['hh_avg_size']  = sensor_panel_2021['S1101_C01_002E']

# 查看结果
sensor_panel_2021.head()

In [ ]:
agg_funcs = {
    'sensor_id': 'nunique',
    'active_days': ['mean', 'std'],
    'active_pct': ['mean', 'std'],
    'pop_total': ['mean', 'std'],
    'pct_hispanic': ['mean', 'std'],
    'pct_white': ['mean', 'std'],
    'pct_bach': ['mean', 'std'],
    'hh_avg_size': ['mean', 'std']
}
summary_table = (
    sensor_panel.groupby('year_period')
    .agg(agg_funcs)
)

summary_table.columns = ['_'.join(col) for col in summary_table.columns]
summary_table.reset_index(inplace=True)